# PARTIE III

# Importation des bibliothèques nécessaires

In [70]:
from utils_pos import get_word_tag, preprocess  
import pandas as pd
from collections import defaultdict
import math
import numpy as np

# Lecture du fichier de vocabulaire

In [71]:
with open("fra_mixed-typical_2012_1M-words.txt", 'r', encoding='utf-8') as f:

    vocabulaire = f.readlines()

In [72]:
voca = []

# parcourir chaque ligne dans le texte
for ligne in vocabulaire:
    # diviser la ligne en trois parties et stocker la deuxième partie
    deuxieme_partie = ligne.split()[1]
    # ajouter la deuxième partie à la liste
    voca.append(deuxieme_partie)

# afficher la liste des deuxièmes parties

print("A few items of the vocabulary list")
print(voca[0:10])
print()
print("A few items at the end of the vocabulary list")
print(voca[-10:])

A few items of the vocabulary list
['!', '"', '#', '$', '%', '&', "'", '(', ')', '*']

A few items at the end of the vocabulary list
['--unk--', '--unk_adj--', '--unk_adv--', '--unk_digit--', '--unk_noun--', '--unk_punct--', '--unk_upper--', '--unk_verb--', '.', '...']


In [73]:
len(voca)

298974

In [74]:
print(voca[100])

vous


In [75]:

voca = list(set(voca))
len(voca)

268123

In [76]:
# vocab: dictionary that has the index of the corresponding words
vocab = {} 

# Get the index of the corresponding words. 
for i, word in enumerate(sorted(voca)): 
    vocab[word] = i       
    
print("Vocabulary dictionary, key is the word, value is a unique integer")
cnt = 0
for k,v in vocab.items():
    print(f"{k}:{v}")
    cnt += 1
    if cnt > 50:
        break

Vocabulary dictionary, key is the word, value is a unique integer
!:0
":1
#:2
$:3
$.:4
$/personne:5
$1:6
$1000:7
$20:8
$400:9
$500:10
$60:11
$600:12
$800k:13
$?:14
$content:15
$contents:16
$lambda$-termes:17
$n:18
$postpone:19
$t:20
%:21
%!:22
%.:23
&:24
':25
(:26
):27
*:28
+:29
,:30
-:31
--:32
--n--:33
--unk--:34
--unk_adj--:35
--unk_adv--:36
--unk_digit--:37
--unk_noun--:38
--unk_punct--:39
--unk_upper--:40
--unk_verb--:41
.:42
...:43
/:44
0:45
0%:46
0''19:47
0''7:48
0'100:49
0,000:50


# Chargement du corpus d'entraînement

In [77]:
with open("tags_train.txt", 'r', encoding='utf-8') as f:
    training = f.read()
    training_corpus = training.split()
print(f"A few items of the training corpus list")
print(training_corpus[10:20])
len(training_corpus)

A few items of the training corpus list
['Nutashkuan_NPP', 'se_CLR', 'sont_V', 'retirées_VPP', 'des_DET', 'négociations_NC', 'territoriales_ADJ', 'avec_P', 'le_DET', 'gouvernement_NC']


2028032

# Nouveau corpus d'entraînement limité

In [78]:
training_corpus = training_corpus[0:33000]
len(training_corpus)

33000

# Chargement du corpus de test

In [79]:
with open("tags_test.txt", 'r', encoding='utf-8') as f:
    test = f.read()
    y = test.split()
print("A sample of the test corpus")
print(y[0:10])

A sample of the test corpus
['Ce_DET', 'vendredi_NC', ',_PONCT', 'quatre_DET', 'matchs_NC', 'de_P', 'la_DET', 'ligue_NC', 'nationale_ADJ', 'de_P']


# Prétraitement des données

# preprocess ()

prend en entrée un vocabulaire et le chemin du fichier de données à prétraiter. Elle lit les données ligne par ligne à partir du fichier, en les séparant en mots. Pour chaque mot, elle vérifie s'il s'agit de la fin d'une phrase (s'il est vide). Si c'est le cas, elle ajoute "--n--" à la liste des mots prétraités prep, qui représente le jeton de fin de phrase. Si le mot n'est pas vide et qu'il n'est pas dans le vocabulaire fourni, elle le remplace par un token inconnu en appelant la fonction assign_unk, puis ajoute ce mot prétraité à la liste prep. Si le mot est présent dans le vocabulaire, elle l'ajoute simplement à la liste prep.

La fonction retourne deux listes : orig qui contient les mots d'origine et prep qui contient les mots prétraités. Les assertions à la fin garantissent que le nombre de mots dans les listes originales et prétraitées correspond au nombre de lignes dans le fichier de données.

In [80]:
#corpus without tags, preprocessed
_, prep = preprocess(vocab, "words.txt")     

print('The length of the preprocessed test corpus: ', len(prep))
print('This is a sample of the test_corpus: ')
print(prep[50:100])

The length of the preprocessed test corpus:  507364
This is a sample of the test_corpus: 
['éventuel', '--unk--', 'ou', 'à', 'des', 'escroqueries', '.', 'Bien', 'que', 'Barack', 'Obama', 'ait', 'obtenu', '61', '%', 'des', 'suffrages', 'en', 'ayant', 'rallié', '--unk_punct--', 'appui', 'de', 'la', 'communauté', 'noire', ',', 'Hillary', 'Clinton', 'a', 'rassemblé', '37', '%', 'des', 'votes', 'pour', 'les', '33', 'délégués', 'de', '--unk_punct--', 'état', '.', 'Ce', 'genre', '--unk_punct--', 'action', 'est', 'inédite', 'dans']


# Caractères de ponctuation

In [81]:
import string
punct = set(string.punctuation)
# Règles de morphologie utilisées pour attribuer des jetons de mot inconnu
noun_suffix = ["action", "age", "ance", "cy", "dom", "ee", "ence", "er", "hood", "ion", "ism", "ist", "ity", "ling", "ment", "ness", "or", "ry", "scape", "ship", "ty"]
verb_suffix = ["ate", "ify", "ise", "ize"]
adj_suffix = ["able", "ese", "ful", "i", "ian", "ible", "ic", "ish", "ive", "less", "ly", "ous"]
adv_suffix = ["ward", "wards", "wise"]

# Assignation de tokens pour les mots inconnus

In [82]:
def assign_unk(tok):
    """
    Assign unknown word tokens
    """
    # Digits
    if any(char.isdigit() for char in tok):
        return "--unk_digit--"

    # Punctuation
    elif any(char in punct for char in tok):
        return "--unk_punct--"

    # Upper-case
    elif any(char.isupper() for char in tok):
        return "--unk_upper--"

    # Nouns
    elif any(tok.endswith(suffix) for suffix in noun_suffix):
        return "--unk_noun--"

    # Verbs
    elif any(tok.endswith(suffix) for suffix in verb_suffix):
        return "--unk_verb--"

    # Adjectives
    elif any(tok.endswith(suffix) for suffix in adj_suffix):
        return "--unk_adj--"

    # Adverbs
    elif any(tok.endswith(suffix) for suffix in adv_suffix):
        return "--unk_adv--"

    return "--unk--"


# Extraction des mots et des tags

In [83]:
def get_word_tag(line, vocab):
    if not line.strip():
        word = "--n--"
        tag = "--s--"
    else:
        a = line.split("_")
        if len(a) == 2:
            word, tag = a
            if word not in vocab: 
                word = assign_unk(word)
        else:
            word = "--n--"
            tag = "--s--"
    return word, tag

In [84]:
get_word_tag('se_CLR', vocab)

('se', 'CLR')

In [85]:
l=[]
for word_tag in training_corpus:
    a = get_word_tag(word_tag, vocab)
    l.append(a)

In [86]:
l[1:5]

[('communautés', 'NC'),
 ('--unk--', 'ADJ'),
 ('--unk_punct--', 'P'),
 ('--unk_upper--', 'NPP')]

# Création des matrices de transition et d'émission

#                                                create_dictionaries ()

                                            
prend en entrée un corpus d'entraînement et un vocabulaire, puis elle crée et renvoie trois dictionnaires : emission_counts, transition_counts, et tag_counts. Ces dictionnaires sont utilisés dans le modèle de Markov caché pour représenter les comptes d'émissions, de transitions et d'étiquettes respectivement. La fonction parcourt chaque paire mot-étiquette dans le corpus d'entraînement, comptant les occurrences de chaque transition, émission et étiquette. Elle renvoie ensuite ces compteurs.

In [87]:
def create_dictionaries(training_corpus, vocab):
    """
    Input: 
        training_corpus: un corpus où chaque ligne contient un mot suivi de son étiquette.
        vocab: un dictionnaire où les clés sont les mots du vocabulaire et la valeur est un indice
    Output: 
        emission_counts: un dictionnaire où les clés sont (étiquette, mot) et les valeurs sont les comptes
        transition_counts: un dictionnaire où les clés sont (prev_tag, tag) et les valeurs sont les comptes
        tag_counts: un dictionnaire où les clés sont les étiquettes et les valeurs sont les comptes
    """
    
    # Initialiser les dictionnaires en utilisant defaultdict
    emission_counts = defaultdict(int)
    transition_counts = defaultdict(int)
    tag_counts = defaultdict(int)
    
    # Initialiser "prev_tag" (étiquette précédente) avec l'état de départ, indiqué par '--s--'
    prev_tag = '--s--' 
    
    # Utiliser 'i' pour suivre le numéro de ligne dans le corpus
    i = 0 
    
    # Chaque élément dans le corpus d'entraînement contient un mot et son étiquette POS
    # Parcourir chaque mot et son étiquette dans le corpus d'entraînement
    for word_tag in training_corpus:
        
        # Incrémenter le compteur de mots et étiquettes
        i += 1
        
        # Tous les 50 000 mots, imprimer le compteur de mots
        if i % 50000 == 0:
            print(f"nombre de mots = {i}")
        # Obtenir le mot et l'étiquette en utilisant la fonction d'aide get_word_tag (importée depuis utils_pos.py)
        word, tag = get_word_tag(word_tag, vocab) 
        
        # Incrémenter le compteur de transition pour le mot et l'étiquette précédents
        transition_counts[(prev_tag, tag)] += 1
        
        # Incrémenter le compteur d'émission pour l'étiquette et le mot
        emission_counts[(tag, word)] += 1

        # Incrémenter le compteur d'étiquettes
        tag_counts[tag] += 1

        # Définir l'étiquette précédente sur cette étiquette (pour la prochaine itération de la boucle)
        prev_tag = tag
        
    return emission_counts, transition_counts, tag_counts


In [88]:
emission_counts, transition_counts, tag_counts = create_dictionaries(training_corpus, vocab)
a= sorted(emission_counts.keys())


In [89]:
# get all the POS states
states = sorted(tag_counts.keys())
print(f"Number of POS tags (number of 'states'): {len(states)}")
print("View these POS tags (states)")
print(states)

Number of POS tags (number of 'states'): 30
View these POS tags (states)
['--s--', 'ADJ', 'ADJWH', 'ADV', 'ADVWH', 'CC', 'CLO', 'CLR', 'CLS', 'CS', 'DET', 'DETWH', 'ET', 'I', 'NC', 'NPP', 'P', 'P+D', 'PONCT', 'PREF', 'PRO', 'PROREL', 'PROWH', 'U', 'V', 'VIMP', 'VINF', 'VPP', 'VPR', 'VS']


In [90]:
print("transition examples: ")
for ex in list(transition_counts.items())[:6]:
    print(ex)
print()

print("emission examples: ")
for ex in list(emission_counts.items())[200:203]:
    print (ex)
print()

print("ambiguous word example: ")
for tup,cnt in emission_counts.items():
    if tup[1] == 'you': print (tup, cnt) 

transition examples: 
(('--s--', 'DET'), 1)
(('DET', 'NC'), 3895)
(('NC', 'ADJ'), 1127)
(('ADJ', 'P'), 431)
(('P', 'NPP'), 433)
(('NPP', 'PONCT'), 821)

emission examples: 
(('ADJ', 'autonome'), 1)
(('NC', 'transports'), 3)
(('ADJ', 'parisiens'), 1)

ambiguous word example: 


#                                               predict_pos ()

                                  
prend en entrée une version prétraitée d'un corpus prep, le corpus étiqueté y, un dictionnaire emission_counts contenant les comptes d'émissions, un vocabulaire vocab, et une liste states de toutes les étiquettes possibles. Elle retourne la précision de la prédiction, c'est-à-dire le nombre de fois où un mot a été correctement classé.

Elle parcourt chaque mot du corpus prétraité et son étiquette associée. Pour chaque mot, elle vérifie s'il est présent dans le vocabulaire. Si oui, elle trouve le POS prédit avec le plus grand nombre de compte d'émission pour ce mot, en utilisant les informations stockées dans emission_counts. Si le POS prédit correspond au vrai POS, elle incrémente le nombre de prédictions correctes. Enfin, elle calcule et retourne la précision en divisant le nombre de prédictions correctes par le nombre total de mots dans le corpus.

In [91]:
def predict_pos(prep, y, emission_counts, vocab, states):
    '''
    Input: 
        prep: une version prétraitée de 'y'. Une liste avec le composant 'mot' des tuples.
        y: un corpus composé d'une liste de tuples où chaque tuple consiste en (mot, POS)
        emission_counts: un dictionnaire où les clés sont des tuples (tag, mot) et la valeur est le compte
        vocab: un dictionnaire où les clés sont des mots dans le vocabulaire et la valeur est un indice
        states: une liste triée de toutes les étiquettes possibles pour cet exercice
    Output: 
        accuracy: nombre de fois où vous avez classé un mot correctement
    '''
    
   
    num_correct = 0
    
    # Obtenir les tuples (tag, mot), stockés sous forme d'ensemble
    all_words = set(emission_counts.keys())
    
    # Obtenir le nombre de tuples (mot, POS) dans le corpus 'y'
    total = len(y)
    for word, y_tup in zip(prep, y): 
        

        # Diviser la chaîne (mot, POS) en une liste de deux éléments
        y_tup_l = y_tup.split('_')
        
        # Vérifier que y_tup contient à la fois le mot et le POS
        if len(y_tup_l) == 2:
            
            # Définir l'étiquette POS réelle pour ce mot
            true_label = y_tup_l[1]

        else:
            # Si y_tup ne contient pas le mot et le POS, passer au mot suivant
            continue
    
        count_final = 0
        pos_final = ''
        
        # Si le mot est dans le vocabulaire...
        if word in vocab:
            for pos in states:
                        
                # définir la clé comme le tuple contenant le POS et le mot
                key = (pos, word)

                # vérifier si la clé (pos, mot) existe dans le dictionnaire emission_counts
                if key in emission_counts: # compléter cette ligne

                # obtenir le compte d'émission du tuple (pos, mot) 
                    count = emission_counts[key]

                    # garder une trace du POS avec le plus grand compte
                    if count > count_final: # compléter cette ligne

                        # mettre à jour le compteur final (plus grand compte)
                        count_final = count

                        # mettre à jour le POS final
                        pos_final = pos

            # Si le POS final (avec le plus grand compte) correspond au vrai POS :
            if pos_final == true_label : # compléter cette ligne
                
                # Mettre à jour le nombre de prédictions correctes
                num_correct += 1
            
  
    accuracy = num_correct / total
    
    return accuracy


In [92]:
accuracy_predict_pos = predict_pos(prep, y, emission_counts, vocab, states)
print(f"Accuracy of prediction using predict_pos is {accuracy_predict_pos:.9f}")

Accuracy of prediction using predict_pos is 0.797116074


# create_transition_matrix ()

prend en entrée le paramètre de lissage alpha, un dictionnaire tag_counts qui mappe chaque étiquette à son nombre respectif, et un dictionnaire transition_counts contenant les comptes de transition pour les étiquettes précédentes et actuelles. Elle retourne la matrice de transition A.

Elle initialise une matrice A de dimensions (num_tags, num_tags) où num_tags est le nombre d'étiquettes POS uniques. Ensuite, elle parcourt chaque paire d'étiquettes POS précédente et actuelle. Pour chaque paire, elle vérifie si elle existe dans les comptes de transition. Si c'est le cas, elle récupère le compte correspondant ; sinon, le compte est initialisé à zéro. Ensuite, elle applique le lissage en utilisant les comptes, alpha, et le nombre total d'étiquettes pour calculer les probabilités de transition et les stocke dans la matrice A. Enfin, elle retourne la matrice de transition ainsi calculée.

In [93]:
def create_transition_matrix(alpha, tag_counts, transition_counts):
    ''' 
    Input: 
        alpha: nombre utilisé pour le lissage
        tag_counts: un dictionnaire qui mappe chaque étiquette à son compte respectif
        transition_counts: compte de transition pour le mot et l'étiquette précédents
    Output:
        A: matrice de dimension (num_tags, num_tags)
    '''
    # Obtenir une liste triée des étiquettes POS uniques
    all_tags = sorted(tag_counts.keys())
    
    # Compter le nombre d'étiquettes POS uniques
    num_tags = len(all_tags)
    
    # Initialiser la matrice de transition 'A'
    A = np.zeros((num_tags, num_tags))
    
    # Obtenir les tuples de transition uniques (POS précédent, POS actuel)
    trans_keys = set(transition_counts.keys())
    
    # Parcourir chaque ligne de la matrice de transition A
    for i in range(num_tags):
        
        # Parcourir chaque colonne de la matrice de transition A
        for j in range(num_tags):

            # Initialiser le compteur du (POS précédent, POS actuel) à zéro
            count = 0
        
            # Définir le tuple (POS précédent, POS actuel)
            # Obtenir l'étiquette à la position i et l'étiquette à la position j (à partir de la liste all_tags)
            key = (all_tags[i], all_tags[j])

            # Vérifier si le tuple (POS précédent, POS actuel) 
            # existe dans le dictionnaire de comptes de transition
            if key in transition_counts: # compléter cette ligne
                # Obtenir le compte à partir du dictionnaire transition_counts 
                # pour le tuple (POS précédent, POS actuel)
                count = transition_counts[key]
                
            # Obtenir le compte de l'étiquette précédente (position d'index i) à partir de tag_counts
            count_prev_tag = tag_counts[all_tags[i]]
            
            # Appliquer le lissage en utilisant le compte du tuple, alpha, 
            # le compte de l'étiquette précédente, alpha, et le nombre total d'étiquettes
            A[i, j] = (count + alpha) / (count_prev_tag + alpha * num_tags)
    
    return A


In [94]:
alpha = 0.001
A = create_transition_matrix(alpha, tag_counts, transition_counts)

print(f"A at row 0, col 0: {A[0,0]:.9f}")
print(f"A at row 3, col 1: {A[3,1]:.4f}")

print("View a subset of transition matrix A")
A_sub = pd.DataFrame(A[10:15,10:15], index=states[10:15], columns = states[10:15] )
print(A_sub)

A at row 0, col 0: 0.000970874
A at row 3, col 1: 0.0857
View a subset of transition matrix A
            DET         DETWH        ET             I        NC
DET    0.001836  2.039555e-07  0.001836  2.039555e-07  0.794407
DETWH  0.000971  9.708738e-04  0.000971  9.708738e-04  0.971845
ET     0.006999  6.991540e-06  0.601279  6.991540e-06  0.027973
I      0.000330  3.300330e-04  0.000330  3.300330e-04  0.000330
NC     0.016346  1.459422e-07  0.000876  1.459422e-07  0.020724


# create_emission_matrix ()

prend en entrée le paramètre de lissage alpha, un dictionnaire tag_counts qui mappe chaque étiquette à son nombre respectif, un dictionnaire emission_counts où les clés sont des tuples (étiquette, mot) et les valeurs sont les comptes, et un dictionnaire vocab où les clés sont les mots du vocabulaire et la valeur est un index. Elle retourne la matrice d'émission B.

Elle initialise une matrice B de dimensions (num_tags, len(vocab)) où num_tags est le nombre d'étiquettes POS uniques et len(vocab) est le nombre total de mots uniques dans le vocabulaire. Ensuite, elle parcourt chaque paire d'étiquette POS et de mot. Pour chaque paire, elle vérifie si elle existe dans les comptes d'émission. Si c'est le cas, elle récupère le compte correspondant ; sinon, le compte est initialisé à zéro. Ensuite, elle applique le lissage en utilisant les comptes, alpha, et le nombre total de mots pour calculer les probabilités d'émission et les stocke dans la matrice B. Enfin, elle retourne la matrice d'émission ainsi calculée.

In [95]:
def create_emission_matrix(alpha, tag_counts, emission_counts, vocab):
    '''
    Input: 
        alpha: paramètre de réglage utilisé dans le lissage 
        tag_counts: un dictionnaire qui mappe chaque étiquette à son compte respectif
        emission_counts: un dictionnaire où les clés sont (étiquette, mot) et les valeurs sont les comptes
        vocab: un dictionnaire où les clés sont les mots du vocabulaire et la valeur est un index
    Output:
        B: une matrice de dimension (num_tags, len(vocab))
    '''  
    # obtenir le nombre d'étiquettes POS
    num_tags = len(tag_counts)
    
    # Obtenez une liste de toutes les étiquettes POS
    all_tags = sorted(tag_counts.keys())
    
    # Obtenez le nombre total de mots uniques dans le vocabulaire
    num_words = len(vocab)
    
    # Initialiser la matrice d'émission B avec des emplacements pour
    # les étiquettes dans les lignes et les mots dans les colonnes
    B = np.zeros((num_tags, num_words))
    
    # Obtenez un ensemble de tous les tuples (POS, mot) 
    # à partir des clés du dictionnaire emission_counts
    emis_keys = set(list(emission_counts.keys()))
    # Parcourir chaque ligne (étiquettes POS)
    for i in range(num_tags): # compléter cette ligne
        
        # Parcourir chaque colonne (mots)
        for j in range(num_words): # compléter cette ligne

            # Initialiser le compteur d'émission pour le (étiquette POS, mot) à zéro
            count = 0
                    
            # Définir le tuple (étiquette POS, mot) pour cette ligne et cette colonne
            key =  (all_tags[i], vocab[j])

            # Vérifier si le tuple (étiquette POS, mot) existe en tant que clé dans emission_counts
            if key in emission_counts.keys(): # compléter cette ligne
        
                # Obtenez le compte de (étiquette POS, mot) à partir du dictionnaire emission_counts
                count = emission_counts[key]
                
            # Obtenez le compte de l'étiquette POS
            count_tag = tag_counts[all_tags[i]]
                
            # Appliquer le lissage et stocker la valeur lissée 
            # dans la matrice d'émission B pour cette ligne et cette colonne
            B[i,j] = (count + alpha) / (count_tag + alpha*num_words)
    return B


In [96]:
vocab['sera']

246918

In [97]:

B = create_emission_matrix(alpha, tag_counts, emission_counts, list(vocab))

print(f"Voir la position de la matrice à la ligne 0, colonne 0 : {B[0,0]:.9f}")
print(f"Voir la position de la matrice à la ligne 3, colonne 1 : {B[3,1]:.9f}")


cidx  = ['sera','nouveau','jour', 'entre', 'architecture']

# Obtenez l'ID entier pour chaque mot
cols = [vocab[a] for a in cidx]
print(cols)

# Choisissez les étiquettes POS à afficher dans un dataframe d'échantillon
rvals =[ 'ADJWH', 'ADV', 'ADVWH', 'CC']

# Pour chaque étiquette POS, obtenez le numéro de ligne à partir de la liste 'states'
rows = [states.index(a) for a in rvals]
print(rows)

# Obtenez les émissions pour l'échantillon de mots et l'échantillon d'étiquettes POS
B_sub = pd.DataFrame(B[np.ix_(rows,cols)], index=rvals, columns=cidx)
print(B_sub)


Voir la position de la matrice à la ligne 0, colonne 0 : 0.000003716
Voir la position de la matrice à la ligne 3, colonne 1 : 0.000000686
[246918, 219451, 194972, 177549, 139333]
[2, 3, 4, 5]
               sera       nouveau          jour         entre  architecture
ADJWH  3.715773e-06  3.715773e-06  3.715773e-06  3.715773e-06  3.715773e-06
ADV    6.858132e-07  6.858132e-07  6.858132e-07  6.858132e-07  6.858132e-07
ADVWH  3.621574e-06  3.621574e-06  3.621574e-06  3.621574e-06  3.621574e-06
CC     1.132345e-06  1.132345e-06  1.132345e-06  1.132345e-06  1.132345e-06


# Initialisation des probabilités de chemin optimal

# initialize ()

prend en entrée une liste states de toutes les parties du discours possibles, un dictionnaire tag_counts associant chaque étiquette à son nombre respectif, une matrice de transition A de dimension (num_tags, num_tags), une matrice d'émission B de dimension (num_tags, len(vocab)), une séquence de mots corpus dont la POS doit être identifiée, et un dictionnaire vocab où les clés sont des mots du vocabulaire et la valeur est un indice.

Elle initialise deux matrices, best_probs et best_paths, de dimensions (num_tags, len(corpus)). La matrice best_probs stocke les meilleures probabilités pour chaque étiquette POS à chaque position dans le corpus, tandis que la matrice best_paths stocke les chemins (c'est-à-dire les étiquettes POS précédentes) qui ont conduit à ces meilleures probabilités.

La fonction parcourt chaque étiquette POS et initialise les probabilités dans best_probs en fonction de la probabilité de transition du jeton de départ vers cette étiquette POS et de la probabilité d'émission du premier mot du corpus. Si la probabilité de transition du jeton de départ vers une étiquette POS est nulle, la probabilité correspondante dans best_probs est initialisée à moins l'infini.

In [98]:
def initialize(states, tag_counts, A, B, corpus, vocab):
    '''
    Entrées : 
        states : une liste de toutes les parties du discours possibles
        tag_counts : un dictionnaire associant chaque étiquette à son nombre respectif
        A : Matrice de transition de dimension (num_tags, num_tags)
        B : Matrice d'émission de dimension (num_tags, len(vocab))
        corpus : une séquence de mots dont la POS doit être identifiée dans une liste 
        vocab : un dictionnaire où les clés sont des mots du vocabulaire et la valeur est un indice
    Sortie :
        best_probs : matrice de dimension (num_tags, len(corpus)) de nombres flottants
        best_paths : matrice de dimension (num_tags, len(corpus)) d'entiers
    '''
   
    num_tags = len(tag_counts)
    
    # Initialiser la matrice best_probs
    # Les étiquettes POS dans les lignes, le nombre de mots dans le corpus comme colonnes
    best_probs = np.zeros((num_tags, len(corpus)))
    
    # Initialiser la matrice best_paths
    # Les étiquettes POS dans les lignes, le nombre de mots dans le corpus comme colonnes
    best_paths = np.zeros((num_tags, len(corpus)), dtype=int)
    
    # Définir le jeton de départ
    s_idx = states.index("--s--")
    
    # Parcourir chacune des étiquettes POS
    for i in range(num_tags):
        
        # Gérer le cas spécial lorsque la transition du jeton de départ à l'étiquette POS i est nulle
        if A[s_idx, i] == 0:
            
            # Initialiser best_probs à l'étiquette POS 'i', colonne 0, à moins l'infini
            best_probs[i, 0] = float('-inf')
        
        # Pour tous les autres cas lorsque la transition du jeton de départ à l'étiquette POS i n'est pas nulle :
        else:
            # Initialiser best_probs à l'étiquette POS 'i', colonne 0
            # Vérifier la formule dans les instructions ci-dessus
            best_probs[i, 0] = math.log(A[s_idx, i]) + math.log(B[i, vocab[corpus[0]]])
    
    return best_probs, best_paths


In [99]:
best_probs, best_paths = initialize(states, tag_counts, A, B, prep, vocab)

In [100]:

print(f"best_probs[0,0]: {best_probs[0,0]:.4f}") 
print(f"best_paths[2,3]: {best_paths[2,3]:.4f}")

best_probs[0,0]: -19.4402
best_paths[2,3]: 0.0000


# Passage en avant de l'algorithme de Viterbi

# viterbi_forward ()

prend en entrée les matrices de transition A et d'émission B, un corpus de test test_corpus, ainsi que deux matrices préalablement initialisées best_probs et best_paths de dimension (num_tags, len(corpus)), et un dictionnaire vocab où les clés sont des mots du vocabulaire et la valeur est un indice.

Elle parcourt chaque mot dans le corpus à partir du deuxième mot (indice 1) et pour chaque étiquette POS possible pour ce mot, elle calcule la probabilité de la meilleure séquence de tags jusqu'à ce mot en utilisant l'algorithme de Viterbi. Pour chaque étiquette POS, elle trouve le chemin (séquence d'étiquettes POS) qui maximise la probabilité jusqu'au mot précédent, puis elle ajoute la probabilité et l'indice de ce chemin dans les matrices best_probs et best_paths pour le mot actuel

In [101]:
def viterbi_forward(A, B, test_corpus, best_probs, best_paths, vocab):
    '''
    Entrée : 
        A, B : les matrices de transition et d'émission respectivement
        test_corpus : une liste contenant un corpus prétraité
        best_probs : une matrice initialisée de dimension (num_tags, len(corpus))
        best_paths : une matrice initialisée de dimension (num_tags, len(corpus))
        vocab : un dictionnaire où les clés sont des mots du vocabulaire et la valeur est un indice 
    Sortie : 
        best_probs : une matrice complétée de dimension (num_tags, len(corpus))
        best_paths : une matrice complétée de dimension (num_tags, len(corpus))
    '''
    
    num_tags = best_probs.shape[0]
    
    # Parcourir chaque mot dans le corpus à partir du mot 1
    # Rappel : le mot 0 a été initialisé dans `initialize()`
    for i in range(1, len(test_corpus)): 
        
        # Afficher le nombre de mots traités, tous les 5000 mots
        if i % 5000 == 0:
            print("Mots traités : {:>8}".format(i))
            
        # Pour chaque étiquette POS unique que le mot actuel peut être
        for j in range(num_tags):
            
            # Initialiser la meilleure probabilité pour le mot i à moins l'infini
            best_prob_i = float("-inf")
            
            # Initialiser le meilleur chemin pour le mot i à None
            best_path_i = None

            # Pour chaque étiquette POS que le mot précédent peut être :
            for k in range(num_tags):
                
                # Calculer la probabilité =
                # meilleures probabilités de l'étiquette POS k, mot précédent i-1 + 
                # log(probabilité de transition de POS k à POS j) + 
                # log(probabilité que l'émission de POS j soit le mot i)
                prob = best_probs[k, i-1] + math.log(A[k, j]) + math.log(B[j, vocab[test_corpus[i]]])

                # Vérifier si la probabilité de ce chemin est supérieure à
                # la meilleure probabilité jusqu'à ce point
                if prob > best_prob_i:
                    
                    # Conserver la meilleure probabilité
                    best_prob_i = prob
                    
                    # Conserver l'étiquette POS du mot précédent
                    # qui fait partie du meilleur chemin.
                    # Enregistrer l'index (entier) associé à
                    # l'étiquette POS de ce mot précédent
                    best_path_i = k

            # Enregistrer la meilleure probabilité pour l'étiquette POS donnée
            # et la position du mot actuel dans le corpus
            best_probs[j, i] = best_prob_i
            
            # Enregistrer l'identifiant unique entier de l'étiquette POS précédente
            # dans la matrice best_paths, pour l'étiquette POS du mot actuel
            # et la position du mot actuel dans le corpus.
            best_paths[j, i] = best_path_i

    return best_probs, best_paths


In [102]:

best_probs, best_paths = viterbi_forward(A, B, prep, best_probs, best_paths, vocab)

Mots traités :     5000
Mots traités :    10000
Mots traités :    15000
Mots traités :    20000
Mots traités :    25000
Mots traités :    30000
Mots traités :    35000
Mots traités :    40000
Mots traités :    45000
Mots traités :    50000
Mots traités :    55000
Mots traités :    60000
Mots traités :    65000
Mots traités :    70000
Mots traités :    75000
Mots traités :    80000
Mots traités :    85000
Mots traités :    90000
Mots traités :    95000
Mots traités :   100000
Mots traités :   105000
Mots traités :   110000
Mots traités :   115000
Mots traités :   120000
Mots traités :   125000
Mots traités :   130000
Mots traités :   135000
Mots traités :   140000
Mots traités :   145000
Mots traités :   150000
Mots traités :   155000
Mots traités :   160000
Mots traités :   165000
Mots traités :   170000
Mots traités :   175000
Mots traités :   180000
Mots traités :   185000
Mots traités :   190000
Mots traités :   195000
Mots traités :   200000
Mots traités :   205000
Mots traités :  

In [103]:
# tester la fonction
print(f"best_probs[0,1]: {best_probs[0,1]:.4f}") 
print(f"best_probs[0,4]: {best_probs[0,4]:.4f}") 

best_probs[0,1]: -33.3522
best_probs[0,4]: -49.1182


# Passage en arrière de l'algorithme de Viterbi

# viterbi_backward

prend en entrée les matrices best_probs et best_paths résultantes de l'algorithme de Viterbi appliqué dans le sens direct, ainsi que le corpus de mots corpus et la liste de toutes les étiquettes possibles states.

Elle retourne le meilleur chemin prédit en parcourant les matrices best_probs et best_paths en sens inverse, à partir du dernier mot dans le corpus jusqu'au premier. Pour chaque mot, elle récupère l'index de l'étiquette POS prédite avec la probabilité la plus élevée à partir de la matrice best_probs et utilise cet index pour récupérer l'étiquette POS prédite à partir de la matrice best_paths. Elle stocke ensuite cette étiquette POS prédite dans le tableau pred.

In [104]:
def viterbi_backward(best_probs, best_paths, corpus, states):
    '''
    Cette fonction retourne le meilleur chemin.
    '''
    # Obtenir le nombre de mots dans le corpus
    # qui est également le nombre de colonnes dans best_probs, best_paths
    m = best_paths.shape[1] 
    
    # Initialiser le tableau z, de la même longueur que le corpus
    z = [None] * m
    
    # Obtenir le nombre d'étiquettes POS uniques
    num_tags = best_probs.shape[0]
    
    # Initialiser la meilleure probabilité pour le dernier mot
    best_prob_for_last_word = float('-inf')
    
    # Initialiser le tableau pred, de la même longueur que le corpus
    pred = [None] * m
    
    # Étape 1
    
    # Parcourir chaque étiquette POS pour le dernier mot (dernière colonne de best_probs)
    # afin de trouver la ligne (identifiant entier de l'étiquette POS) 
    # avec la probabilité la plus élevée pour le dernier mot
    for k in range(num_tags):
        
        # Si la probabilité de l'étiquette POS à la ligne k 
        # est meilleure que la meilleure probabilité précédente pour le dernier mot :
        if best_probs[k, -1] > best_prob_for_last_word:
            
            # Stocker la nouvelle meilleure probabilité pour le dernier mot
            best_prob_for_last_word = best_probs[k, -1]
    
            # Stocker l'identifiant unique de l'étiquette POS
            # qui est également le numéro de ligne dans best_probs
            z[m - 1] = k
            
    # Convertir l'étiquette POS prédite du dernier mot
    # de son identifiant entier unique en la représentation sous forme de chaîne
    # en utilisant le dictionnaire 'states'
    # stocker ceci dans le tableau 'pred' pour le dernier mot
    pred[m - 1] = states[k]
    
    # Étape 2
    
    # Trouver les meilleures étiquettes POS en parcourant en arrière les best_paths
    # Du dernier mot dans le corpus au 0ème mot dans le corpus
    for i in range(len(corpus) - 1, -1, -1):
        
        # Récupérer l'identifiant entier unique de
        # l'étiquette POS pour le mot à la position 'i' dans le corpus
        pos_tag_for_word_i = best_paths[np.argmax(best_probs[:, i]), i]
        
        # Dans best_paths, aller à la ligne représentant l'étiquette POS du mot i
        # et la colonne représentant la position du mot dans le corpus
        # pour récupérer l'étiquette POS prédite pour le mot à la position i-1 dans le corpus
        z[i - 1] = best_paths[pos_tag_for_word_i, i]
        
        # Obtenir l'étiquette POS du mot précédent sous forme de chaîne
        # Utiliser le dictionnaire 'states',
        # où la clé est l'identifiant entier unique de l'étiquette POS,
        # et la valeur est la représentation de chaîne de cette étiquette POS
        pred[i - 1] = states[pos_tag_for_word_i]
        
    return pred


In [105]:

pred = viterbi_backward(best_probs, best_paths, prep, states)
m=len(pred)
print('The prediction for pred[-7:m-1] is: \n', prep[-7:m-1], "\n", pred[-7:m-1], "\n")
print('The prediction for pred[0:8] is: \n', pred[0:7], "\n", prep[0:7])

The prediction for pred[-7:m-1] is: 
 ['hauteur', 'se', 'sont', 'échappées', 'du', 'véhicule'] 
 ['NC', 'CLR', 'V', 'VPP', 'P+D', 'NC'] 

The prediction for pred[0:8] is: 
 ['DET', 'NC', 'PONCT', 'DET', 'NC', 'P', 'DET'] 
 ['Ce', 'vendredi', ',', 'quatre', 'matchs', 'de', 'la']


In [106]:
print('The third word is:', prep[3])
print('Your prediction is:', pred[3])
print('Your corresponding label y is: ', y[3])

The third word is: quatre
Your prediction is: DET
Your corresponding label y is:  quatre_DET


# Calcul de l'exactitude

# compute_accuracy ()

prend en entrée deux listes : pred, contenant les parties du discours prédites, et y, contenant les lignes où chaque mot est suivi de son étiquette (séparés par un '\t').

Elle calcule le taux de précision en comparant les parties du discours prédites avec les étiquettes réelles. Pour chaque prédiction et étiquette réelle, elle vérifie si elles correspondent, puis incrémente le compteur num_correct en conséquence. Elle compte également le nombre total d'exemples valides avec des étiquettes. Enfin, elle calcule le taux de précision en divisant le nombre de prédictions correctes par le nombre total d'exemples valides et retourne ce taux de précision.

In [107]:
def compute_accuracy(pred, y):
    '''
    Entrée : 
        pred : une liste des parties du discours prédites
        y : une liste de lignes où chaque mot est séparé par un '\t' (c'est-à-dire mot \t étiquette)
    Sortie : 
        accuracy : le taux de précision calculé comme le nombre de prédictions correctes divisé par le nombre total de prédictions valides
    '''
    num_correct = 0
    total = 0
    
    # Combinez ensemble la prédiction et les étiquettes
    for prediction, y in zip(pred, y):
        
        # Séparez l'étiquette en mot et en étiquette POS
        mot_etiquette_tuple = y.split('_')
        
        # Vérifiez s'il y a effectivement un mot et une étiquette
        # ni plus ni moins de 2 éléments
        if len(mot_etiquette_tuple) != 2:
            continue 

        # Stockez le mot et l'étiquette séparément
        mot, etiquet = mot_etiquette_tuple
        
        # Vérifiez si l'étiquette POS correspond à la prédiction
        if prediction == etiquet:
            
            # Comptez le nombre de fois où la prédiction
            # et l'étiquette correspondent
            num_correct += 1
            
        # Gardez une trace du nombre total d'exemples (ayant des étiquettes valides)
        total += 1
        
   
    accuracy = num_correct / total
    
    return accuracy


# Accuracy 

In [108]:
print(f"Accuracy of the Viterbi algorithm is {compute_accuracy(pred, y):.4f}")

Accuracy of the Viterbi algorithm is 0.8863
